In [1]:
import pandas as pd
import numpy as np
import os

cur_dir = os.getcwd()
dir_list = os.listdir()
data_list = os.listdir(cur_dir)

In [2]:
data_dir = cur_dir + '/데이터'
data_list = os.listdir(data_dir)
print("데이터셋 목록(data_list) : ", data_list)

데이터셋 목록(data_list) :  ['learning_action.csv', 'learning_time.csv', 'member.csv']


In [3]:
Dataset = dict()

for idx, data in enumerate(data_list):
    print('불러오는 데이터: ', data)
    Dataset[data[:-4]] = pd.read_csv(cur_dir +f'/데이터/{data}', index_col=0, encoding='cp949').reset_index()
    #[:-4]인덱싱해서 '.CSV' 잘라줌

불러오는 데이터:  learning_action.csv
불러오는 데이터:  learning_time.csv
불러오는 데이터:  member.csv


In [4]:
#데이터 불러오기
df1 = Dataset['member']
df2 = Dataset['learning_action']
df3 = Dataset['learning_time']

In [5]:
#데이터 합치기
df = pd.merge(df1, df2, how = 'left', left_on = ['userid'], right_on= ['userid'])
df = pd.merge(df, df3, how = 'left', left_on = ['userid', 'proc_ymd'], right_on = ['userid', 'proc_ymd'])

In [6]:
df.shape

(502, 36)

## 결측치 제거

In [7]:
#결측치가 100퍼센트인 playtime_sum 지우기
df.drop('playtime_sum', axis=1, inplace=True)

#필요없는 index 열 제거
df.drop('index_x', axis=1, inplace=True)
df.drop('index_y', axis=1, inplace=True)

In [8]:
# 중복값 제거
df_no_duplicates = df.drop_duplicates()

# 'userid' 열을 기준으로 중복값 제거
df_no_duplicates = df.drop_duplicates(subset='userid')

In [9]:
#'assessment_completed_yn', 'media_completed_yn' 둘다 완료일 경우 'completed_yn'완료
df['completed_yn'] = df.apply(lambda row: 'Y' if (row['assessment_completed_yn'] == 'Y') and (row['media_completed_yn'] == 'Y') else 'N', axis=1)

In [10]:
#학습시간이 1시간 이하인 것만
df = df[df['system_learning_time'] <= 3600]

In [11]:
# 결측치가 5개 이상인 행 제거
df = df.dropna(axis = 0, thresh = 30)

In [12]:
#'restart_study' 평균값으로 대체
means = df['restart_study'].mean()
df['restart_study'] = df['restart_study'].fillna(means)

#'score' 평균값으로 대체
means = df['score'].mean()
df['score'] = df['score'].fillna(means)

In [13]:
#값이 0인것 확인
col_list = df.columns.to_series()

for idx, col in enumerate(col_list):
    print(idx, col,'\n', df[df[col]==0])
    print('\n','구분선')

0 userid 
 Empty DataFrame
Columns: [userid, biz_sect_cd_nm, mbr_sex_cd_nm, mbr_step_cd_nm, user_grade_nm, mbr_status_cd_nm, grade_sect_cd, status, re_purch, proc_ymd, media_action_cnt, non_video_viewed_cnt, start_cnt, restart_video, pause_cnt, jump_cnt, resume_cnt, speed_cnt, volume_cnt, end_cnt, index, mcode, completed_yn, assessment_completed_yn, media_completed_yn, system_learning_time, restart_study, score, item_cnt, quizcode_cnt, correct_cnt, solved_time_sum, solved_time_avg]
Index: []

[0 rows x 33 columns]

 구분선
1 biz_sect_cd_nm 
 Empty DataFrame
Columns: [userid, biz_sect_cd_nm, mbr_sex_cd_nm, mbr_step_cd_nm, user_grade_nm, mbr_status_cd_nm, grade_sect_cd, status, re_purch, proc_ymd, media_action_cnt, non_video_viewed_cnt, start_cnt, restart_video, pause_cnt, jump_cnt, resume_cnt, speed_cnt, volume_cnt, end_cnt, index, mcode, completed_yn, assessment_completed_yn, media_completed_yn, system_learning_time, restart_study, score, item_cnt, quizcode_cnt, correct_cnt, solved_time_s

### 미디어 학습이 비디오뿐인가?
그렇다면 밑에꺼 진행
아니면 안해도됨

In [34]:
#start_cnt(비디오시작횟수) 없는데 미디어학습 0이 아닌 것 확인
non_start = df.loc[(df['start_cnt'] == 0) & (df['media_action_cnt'] != 0)]

#미디어학습 없는데 start_cnt(비디오시작횟수) 있는 것 확인 (실행결과>>없음)
non_start2 = df.loc[(df['start_cnt'] != 0) & (df['media_action_cnt'] == 0)]

non_start[['userid', 'media_action_cnt']]

,userid,media_action_cnt
1,91745902-cd00-4e9d-910e-d4f3101adf21,1
2,88efda5e-cd2e-4b7b-8a06-7b0acb67a884,1
4,2eb9a149-90e1-4ef3-85f7-90553d15d125,2
7,0d164297-f34d-4142-8f5a-eb6c3d45744b,2
8,e057d06f-afe5-4fcc-b2c7-ef2391f10ca7,2
...,...,...
484,6d96c2fd-9665-420d-9377-6d7ef9e0a0a3,4
485,8910c419-36d2-409f-a994-aa7d7eebf00c,15
486,dad93565-58fc-455b-bc2f-1a55f16bea07,1
487,a688588a-1f14-447b-aa45-28181f905f90,1


## 여기부터 하면됩니다

In [ ]:
#비디오 시작+재시작 횟수보다 종료횟수가 많은 경우 종료횟수값을 시작+재시작 횟수로 변경
df.loc[df['start_cnt']+df['restart_video'] < df['end_cnt'], 'end_cnt'] = df['start_cnt']+df['restart_video']

In [19]:
# 미디어 또는 비미디어 학습 했으나 시간이 0인 행
err = df.loc[(df['media_action_cnt'] != 0) & (df['non_video_viewed_cnt'] != 0) & (df['system_learning_time'] == 0)]

# 미디어 또는 비미디어 학습 안했으나 시간이 0이 아닌 행 (실행결과>>없음)
err2 = df.loc[(df['media_action_cnt'] == 0) & (df['non_video_viewed_cnt'] == 0) & (df['system_learning_time'] != 0)]

# 해당 행의 'system_learning_time' 값을 평균값으로 변경
if not err.empty:
    mean_system_learning_time = df.loc[df['system_learning_time'] != 0, 'system_learning_time'].mean()
    df.loc[err.index, 'system_learning_time'] = mean_system_learning_time

In [14]:
#풀이문제수가 문제수보다 많은 경우 'item_cnt' 값을 'quizcode_cnt'로 변경
df.loc[df['item_cnt'] > df['quizcode_cnt'], 'item_cnt'] = df['quizcode_cnt']

#정답수가 문제수보다 많은 경우 'correct_cnt' 값을 'quizcode_cnt'로 변경
df.loc[df['correct_cnt'] > df['quizcode_cnt'], 'correct_cnt'] = df['quizcode_cnt']

#정답수가 풀이문제수보다 많은 경우 'correct_cnt' 값을 'item_cnt'로 변경
df.loc[df['correct_cnt'] > df['item_cnt'], 'correct_cnt'] = df['item_cnt']


# 'item_cnt' 결측치인 경우에 'correct_cnt' 값으로 채우기
df['item_cnt'].fillna(df['correct_cnt'], inplace=True)

#'quizcode_cnt' 결측치인 경우에 'item_cnt' 값으로 채우기
df['quizcode_cnt'].fillna(df['item_cnt'], inplace=True)

#'correct_cnt' 결측치인 경우에 'score'값으로 'quizcode_cnt'에서 맞춘갯수 구하여 넣기
df['correct_cnt'].fillna((df['score'] / 100) * df['quizcode_cnt'], inplace=True)

In [15]:
#'solved_time_sum'결측치인 경우에 해당 행의 'item_cnt' 값과 같은 값을 가진 행들의 'solved_time_sum' 평균값으로 채우기
for index, row in df[df['solved_time_sum'].isnull()].iterrows():
    item_cnt_value = row['item_cnt']
    mean_solved_time = df[df['item_cnt'] == item_cnt_value]['solved_time_sum'].mean()
    df.at[index, 'solved_time_sum'] = mean_solved_time

In [16]:
#solved_time_sum 결측치에 solved_time_avg*item_cnt(평균시간*풀이문항 수)
df['solved_time_sum'].fillna(df['solved_time_avg'] * df['item_cnt'], inplace=True)

#solved_time_avg 결측치에 solved_time_sum/item_cnt(시간합계/풀이문항 수)
df['solved_time_avg'].fillna(df['solved_time_sum']/df['item_cnt'], inplace=True)

In [17]:
#null 갯수
null_cnt_df = pd.DataFrame(df.isnull().sum()).rename(columns = {0:'null_count'}).reset_index()

#null 비율
null_cnt_df['null_ratio'] = round(null_cnt_df['null_count']/len(df)*100, 2)

null_cnt_df

,index,null_count,null_ratio
0,userid,0,0.00
1,biz_sect_cd_nm,0,0.00
2,mbr_sex_cd_nm,0,0.00
3,mbr_step_cd_nm,0,0.00
4,user_grade_nm,0,0.00
5,mbr_status_cd_nm,0,0.00
6,grade_sect_cd,0,0.00
7,status,0,0.00
8,re_purch,0,0.00
9,proc_ymd,0,0.00


In [18]:
#행을 기준으로 결측치가 1개라도 있으면 제거
df = df.dropna(axis = 0, how = 'any')

In [19]:
df.shape

(403, 33)

In [20]:
#필요없는 지울 열 선택 (초등, 유료, 사용 - 모든 행이 같은 값)
drop_col = ['biz_sect_cd_nm', 'mbr_step_cd_nm', 'mbr_status_cd_nm']

#필요없는 열 제거
df.drop(drop_col, axis=1, inplace=True)

In [21]:
df.dtypes

userid                      object
mbr_sex_cd_nm               object
user_grade_nm               object
grade_sect_cd               object
status                       int64
re_purch                      bool
proc_ymd                     int64
media_action_cnt             int64
non_video_viewed_cnt         int64
start_cnt                    int64
restart_video                int64
pause_cnt                    int64
jump_cnt                     int64
resume_cnt                   int64
speed_cnt                    int64
volume_cnt                   int64
end_cnt                      int64
index                        int64
mcode                       object
completed_yn                object
assessment_completed_yn     object
media_completed_yn          object
system_learning_time       float64
restart_study              float64
score                      float64
item_cnt                   float64
quizcode_cnt               float64
correct_cnt                float64
solved_time_sum     

In [22]:
# 'restart_study' 열을 반올림
df['restart_study'] = df['restart_study'].round(0)

# 'score' 열을 반올림
df['score'] = df['score'].round(0)

# 'correct_cnt' 열을 반올림
df['correct_cnt'] = df['correct_cnt'].round(0)

In [25]:
df.shape

(403, 30)

In [23]:
# 파일로 저장
df.to_csv('초등이탈데이터_최종.csv', index=False, encoding='utf-8-sig')